<a href="https://colab.research.google.com/github/qinterest/scripts/blob/master/walk_around_UN_Ave.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Let's walk around United Nations Avenue by Python together.

## Setup libraries
Authenticate and import as necessary.

In [0]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

In [0]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

In [4]:
# Folium setup.
import folium
print(folium.__version__)

0.8.3


## Find the UN Ave first

In [0]:
location = [-1.231,36.813]
map = folium.Map(location=location,tiles='Stamen Toner',zoom_start=15)
folium.Marker(
    location,
    popup='UN_Ave'
).add_to(map)
map

## Let's view UN Ave are from space

In [73]:
# Use Landsat 8 surface reflectance data.
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

# Cloud masking function.
opticalBands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
thermalBands = ['B10', 'B11']
def maskL8sr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask1 = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  mask2 = image.mask().reduce('min')
  mask3 = image.select(opticalBands).gt(0).And(
          image.select(opticalBands).lt(10000)).reduce('min')
  mask = mask1.And(mask2).And(mask3)
  return image.select(opticalBands).divide(10000).addBands(
          image.select(thermalBands).divide(10).clamp(273.15, 373.15)
            .subtract(273.15).divide(100)).updateMask(mask)

# The image input data is a cloud-masked median composite.
image = l8sr.filterDate('2015-01-01', '2017-12-31').map(maskL8sr).median()

# Use folium to visualize the imagery.
mapid = image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})


map = folium.Map(location=location,tiles='Stamen Terrain',zoom_start=13)
folium.Marker(
    location,
    popup='UN_Ave'
).add_to(map)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='median composite',
  ).add_to(map)
map

## Let's find vegetation area.

In [70]:
def NDVI(image):
  nir = image.select('B5')
  red = image.select('B4')
  ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')

  return ndvi

mapid = NDVI(image).getMapId({'bands': ['NDVI'], 'min': 0.5, 'max': 1, 'palette':['white','green']})

map = folium.Map(location=location,tiles='Stamen Terrain',zoom_start=13)
folium.Marker(
    location,
    popup='UN_Ave'
).add_to(map)

folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='NDVI',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

## Where's the hot?

In [76]:
map = folium.Map(location=location,tiles='Stamen Terrain',zoom_start=13)
mapid = image.getMapId({'bands': ['B10'], 'min': 0.2, 'max': 0.3,'palette': ['blue','white','#ff0000', '#FFFF00']})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='thermal',
  ).add_to(map)
folium.Marker(
    location,
    popup='UN_Ave'
).add_to(map)
map.add_child(folium.LayerControl())
map

## Let's run a machine learning model to do land classification.

In [86]:
region = ee.Geometry.Rectangle([36.77, -1.26, 36.88, -1.20])

training = image.sample(
    region= region,
    scale=30,
    numPixels=500
);

clusterer = ee.Clusterer.wekaKMeans(3).train(training)
result = image.cluster(clusterer)
cluster_img = result.randomVisualizer()


map = folium.Map(location=location,tiles='Stamen Terrain',zoom_start=13)
mapid = cluster_img.getMapId()
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='cluster',
  ).add_to(map)
folium.Marker(
    location,
    popup='UN_Ave'
).add_to(map)
map.add_child(folium.LayerControl())
map